# Job Queue Management with AiiDA WorkGraph

Suppose you have a workflow that needs to launch multiple subprocesses (e.g., 100 calcjobs), but your computer has resource limitations that only allow running 5 calcjobs concurrently. Additionally, you don't have access to a job management system like SLURM locally. In such cases, you can set up a simple job queue management system using AiiDA's WorkGraph by limiting the maximum number of subprocesses running simultaneously within the WorkGraph. 

## Setting Maximum Number of Running Jobs

You can control the maximum number of running subprocesses inside a WorkGraph by using the following configuration:

```python
wt.max_number_jobs = 5
```

This ensures that no more than 5 subprocesses will be running concurrently within the WorkGraph.

## Example Usage

Let's walk through an example where we create a simple WorkGraph to launch 20 calcjobs.

1. Load the AiiDA profile:


In [1]:
%load_ext aiida
from aiida import load_profile
load_profile()

Profile<uuid='57ccbf7d9e2b41b39edb2bfdaf725feb' name='default'>

2. Create the WorkGraph and submit jobs:


In [2]:
from aiida_workgraph import WorkGraph, build_node
from aiida.orm import load_code
from aiida.orm import Int

# Use the calcjob: ArithmeticAddCalculation
arithmetic_add = build_node("aiida.calculations.arithmetic.add.ArithmeticAddCalculation")
code = load_code("add@localhost")

wt = WorkGraph("test_max_number_jobs")
N = 20
# Create N nodes
for i in range(N):
    temp = wt.nodes.new(arithmetic_add, name=f"add{i}", x=Int(1), y=Int(1), code=code)
    # Set a sleep option for each job (e.g., 10 seconds per job)
    temp.set({"metadata.options.sleep": 10})

# Set the maximum number of running jobs inside the WorkGraph
wt.max_number_jobs = 5
wt.submit(wait=True)

WorkGraph node crated, PK: 2782


3. After submission, you can check the progress of the WorkGraph using the following command:

   ```shell
   verdi process list
   ```

   This command will display information about the currently running calcjobs, including the fact that there are a maximum of 5 calcjobs running simultaneously.

4. Additionally, you can monitor the progress visually by visiting `http://127.0.0.1:8000/workgraph`. Here, you can observe the duration of each node's execution over time.  For the moment, one needs zoom and pan the time scale to see the time bar.


<img width=800 src="../_static/images/wait_job_queue.png" />

By following this approach, you can control the execution of multiple calcjobs within your AiiDA workflow, even without access to a job management system.